In [1]:
import json
from IPython.display import display
# display(pd.Dataframe(data))
import pandas as pd
import numpy as np
import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
from math import pi
from pprint import pprint
from konlpy.tag import Okt
from collections import Counter
from sklearn.manifold import TSNE
from future.utils import iteritems
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import linear_kernel, cosine_similarity

In [2]:
import csv

raw_keyword = ['글로벌역량', '능동', '도전', '성실', '소통', '인내심', '정직', '주인의식', '창의', '팀워크']

# tf-idf값 불러오는 과정
X = pd.read_csv('dataset/results/large_vector_X.csv', header=None)
X = np.array(X)

print(X.shape)
print(type(X))

# 학습 단어(feature) 불러오는 과정
f = open('dataset/results/large_vector_X_features.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
features = list()
    

(10, 1783416)
<class 'numpy.ndarray'>


In [5]:
for i in reader:
    features.append(i[1])

In [6]:
print(features[0:100])

['가가 가격', '가가 가지다', '가가 걱정', '가가 건립', '가가 계약', '가가 공복', '가가 공사', '가가 관건', '가가 광진구', '가가 그린', '가가 근대', '가가 나오다', '가가 농업', '가가 높아지다', '가가 대폭', '가가 되다', '가가 디자인', '가가 만들다', '가가 무대', '가가 무역', '가가 미국', '가가 미비', '가가 밀라노', '가가 벌이', '가가 보시', '가가 보아', '가가 보유', '가가 부잣집', '가가 분양', '가가 뽑다', '가가 사설', '가가 설비', '가가 소유', '가가 수도권', '가가 스타', '가가 스타일', '가가 시세', '가가 신뢰', '가가 앞서다', '가가 업다', '가가 연극', '가가 영화', '가가 오다', '가가 요동치다', '가가 운동', '가가 위해', '가가 의무', '가가 저지르다', '가가 적용', '가가 제품', '가가 조합', '가가 주식', '가가 직접', '가가 차지', '가가 추가', '가가 토니', '가가 특별', '가가 패피', '가가 평소', '가가 하다', '가가 한미', '가가호호', '가가호호 부모님', '가가호호 소통', '가감', '가감 개진', '가감 경제정책', '가감 고백', '가감 공개', '가감 그대로', '가감 뉴스', '가감 답변', '가감 대화', '가감 드러내다', '가감 라디오스타', '가감 만남', '가감 매력', '가감 반응', '가감 보여주다', '가감 선보이다', '가감 스타', '가감 쓰다', '가감 연지', '가감 오가다', '가감 이슈', '가감 이야기', '가감 이행', '가감 전달', '가감 전해', '가감 조정', '가감 털다', '가감 토크', '가감 판박이', '가감 하다', '가객', '가객 공연', '가객 김윤령', '가객 독창', '가객 추다', '가거도']


In [7]:
# 불용어리스트 불러오는 과정
f = open('dataset/stopwords/stopwords.csv', 'r', encoding='utf-8')
reader = csv.reader(f)
stopwords = list()

for row in reader:
    stopwords.append(row[0])

print(type(features))


okt = Okt()

def read_data(filename):
    with open(filename, 'r') as f:
        data = f.read()
        # remove_quotes
        data = data.replace("‘", " ")
        data = data.replace("’", " ")
        data = data.replace("“", " ")
        data = data.replace("”", " ")
        data = data.replace("`", " ")
        data = data.replace("\'", " ")
        data = data.replace("\"", " ")
    return data

def tokenizer(raw, pos=["Noun","Verb"], stopword=stopwords):
    return [
        word for word, tag in okt.pos(
            raw, 
            norm=True,   # normalize 정제 과정
            stem=True    # stemming 정제 과정
            )
            if len(word) > 1 and tag in pos and word not in stopword
        ]


<class 'list'>


In [ ]:
print(features[0:100])

In [ ]:
# 여기부터 기업별 분석 진행.
import json
import os
import glob
from pprint import pprint
from collections import Counter

def read_data(filename):
    with open(filename, 'r', encoding='utf-8') as f:
        json_data = json.load(f)
        one_data = [json_data[i] for i in json_data]
    return one_data
    
print('continue...')
with open('dataset/resumes/educe_SAMSUNG_2385.json', encoding='utf-8') as json_file:
    json_data = json.load(json_file)
print('continue......')
resume_data = ''

for i in json_data:
    resume_data += i['document']
    
print(len(resume_data))

srch=[t for t in tokenizer(resume_data) if t in features]

print(len(srch))


In [ ]:
print(srch[0:100])

In [ ]:
# 분석할때에는 X대신 X.toarray()를 넣어준다
# document term matrix 에서 검색하고자 하는 feature만 뽑아낸다.

srch_dtm = np.asarray(X)[:, [
    # vectorize.vocabulary_.get 는 특정 feature에 가지고 있는 index값을 리턴한다 (features에 단어와 일치하는 것)
    features.index(i) for i in srch
]]


data = pd.DataFrame(srch_dtm, columns=srch)
srch_dtm = data.rename({0:"글로벌역량", 1:"능동", 2:"도전", 3:"성실", 4:"소통", 5:"인내심", 6:"정직", 7:"주인의식", 8:"창의", 9:"팀워크"}, axis='index')
display(pd.DataFrame(srch_dtm))


score = srch_dtm.sum(axis=1)
# score는 문장별 feature 합계 점수
 
for i in score.argsort()[::-1]:
    if score[i] >= 0:
        print('{} // score : {}'.format(raw_keyword[i], score[i]))

In [ ]:
data.to_csv("SAMSUNG_2385_dataframe.csv", mode='w')

In [ ]:
csv_file_score = csv.writer(open("SAMSUNG_score.csv", "w", newline=''))

In [ ]:
csv_file_score.writerow(["score"])

In [ ]:
csv_file_score.writerow([score.글로벌역량])
csv_file_score.writerow([score.능동])
csv_file_score.writerow([score.도전])
csv_file_score.writerow([score.성실])
csv_file_score.writerow([score.소통])
csv_file_score.writerow([score.인내심])
csv_file_score.writerow([score.정직])
csv_file_score.writerow([score.주인의식])
csv_file_score.writerow([score.창의])
csv_file_score.writerow([score.팀워크])

In [4]:

# 검색 문장에서 feature를 뽑아냄
resume_data = read_data('/home/leesunhong/similarity/dataset/input_resume.txt')
a = tokenizer(resume_data)
print(a)

srch=[t for t in tokenizer(resume_data) if t in features]

print(srch)

# 분석할때에는 X대신 X.toarray()를 넣어준다
# document term matrix 에서 검색하고자 하는 feature만 뽑아낸다.

srch_dtm = np.asarray(X)[:, [
    # vectorize.vocabulary_.get 는 특정 feature에 가지고 있는 index값을 리턴한다 (features에 단어와 일치하는 것)
    features.index(i) for i in srch
]]


'''
srch_dtm = np.asarray(X.toarray())[:, [
    # vectorize.vocabulary_.get 는 특정 feature에 가지고 있는 index값을 리턴한다 (features에 단어와 일치하는 것)
    vectorize.vocabulary_.get(i) for i in srch
]]
'''

print(srch_dtm.shape)

data = pd.DataFrame(srch_dtm, columns=srch)
srch_dtm = data.rename({0:"글로벌역량", 1:"능동", 2:"도전", 3:"성실", 4:"소통", 5:"인내심", 6:"정직", 7:"주인의식", 8:"창의", 9:"팀워크"}, axis='index')
display(pd.DataFrame(srch_dtm))


score = srch_dtm.sum(axis=1)
# score는 문장별 feature 합계 점수
 
for i in score.argsort()[::-1]:
    if score[i] >= 0:
        print('{} // score : {}'.format(raw_keyword[i], score[i]))


FileNotFoundError: [Errno 2] No such file or directory: '/home/leesunhong/similarity/dataset/input_resume.txt'